In [ ]:
# %% SOY — ex-ante od 2010 (h=1..4), benchmark persistence, GridSearch MAE
# + rolling ex-post dla ostatnich 6 miesiecy (h=1..6), + prognoza do przodu z czerwcowego anchoru
import warnings
warnings.filterwarnings("ignore")

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dateutil.relativedelta import relativedelta

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.metrics import r2_score

# --------------------------- Parametry ---------------------------
IN_MERGED = r"D:\git20\geo-toolbox\data_out\market_with_enso_proxy_monthly.csv"
PLOT_START = pd.Timestamp("2020-01-01")
HORIZONS_EXANTE = [1, 2, 3, 4]
HORIZONS_RECENT = [1, 2, 3, 4, 5, 6]
LAST_K_ANCHORS = 6
SOY_LAGS = list(range(0, 7))
EXOG_LAGS = list(range(0, 7))
ANCHOR_STEP_MONTHS = 6
WANTED_EXOG = []

# --------------------------- Utils ---------------------------
def winsorize_series(s: pd.Series, lower_q=0.005, upper_q=0.995):
    s_num = pd.to_numeric(s, errors="coerce")
    if s_num.dropna().empty:
        return s
    lo, hi = s_num.quantile(lower_q), s_num.quantile(upper_q)
    return s_num.clip(lo, hi)

def clean_column(series: pd.Series, is_price_like: bool = True) -> pd.Series:
    s = series.copy()
    s = s.interpolate(method="time") if s.index.inferred_type in ("datetime64", "datetime64tz") else s.interpolate()
    s = s.bfill().ffill()
    return winsorize_series(s, 0.005, 0.995)

def add_seasonality_month(df_idx: pd.DatetimeIndex) -> pd.DataFrame:
    m = df_idx.month
    return pd.DataFrame({
        "month_sin": np.sin(2 * np.pi * m / 12.0),
        "month_cos": np.cos(2 * np.pi * m / 12.0),
    }, index=df_idx)

def add_simple_tech_features(s: pd.Series) -> pd.DataFrame:
    df_feat = pd.DataFrame(index=s.index)
    df_feat["SOY_ret_1m"] = s.pct_change(1)
    df_feat["SOY_sma_3m"] = s.rolling(3).mean()
    df_feat["SOY_vol_3m"] = s.pct_change().rolling(3).std()
    for c in df_feat.columns:
        df_feat[c] = clean_column(df_feat[c], is_price_like=False)
    return df_feat

def mae(a, b):
    a = np.asarray(a, float); b = np.asarray(b, float)
    return float(np.nanmean(np.abs(a - b)))

def last_complete_month(ts: pd.Timestamp) -> pd.Timestamp:
    return (ts.replace(day=1) - pd.offsets.Day(1)).normalize()

# --------------------------- Dane ---------------------------
df = pd.read_csv(IN_MERGED, parse_dates=["date"]).set_index("date").sort_index()
df.index = df.index.to_period("M").to_timestamp("M")

drop_cols = {"BG_PRICE", "ADM_PRICE", "SOY_PRICE", "ADM_IDX", "BG_IDX", "SOY_IDX" }
WANTED_EXOG = [c for c in df.columns if c not in drop_cols and pd.api.types.is_numeric_dtype(df[c])]
df["SOY"] = clean_column(df["SOY_PRICE"], is_price_like=True)
na_frac = df.isna().mean()
to_drop = na_frac[na_frac > 0.40].index.difference(["SOY"])
df = df.drop(columns=list(to_drop)) if len(to_drop) > 0 else df
for col in df.columns:
    if col != "SOY":
        df[col] = clean_column(df[col], is_price_like=False)

df_season = add_seasonality_month(df.index)
df_tech = add_simple_tech_features(df["SOY"])
df = pd.concat([df, df_season, df_tech], axis=1)

for L in SOY_LAGS:
    df[f"SOY_lag{L}"] = df["SOY"].shift(L)
exog_raw = [c for c in WANTED_EXOG if c in df.columns]
for c in exog_raw:
    for L in EXOG_LAGS:
        df[f"{c}_lag{L}"] = df[c].shift(L)
for h in sorted(set(HORIZONS_EXANTE) | set(HORIZONS_RECENT)):
    df[f"SOY_t+{h}"] = df["SOY"].shift(-h)

feature_cols = (
    [c for c in df.columns if c.startswith("SOY_lag")] +
    [f"{c}_lag{L}" for c in exog_raw for L in EXOG_LAGS] +
    ["month_sin", "month_cos", "SOY_ret_1m", "SOY_sma_3m", "SOY_vol_3m"]
)
feature_cols = [c for c in feature_cols if c in df.columns]

# ---------------------- Model ----------------------
pre = ColumnTransformer([
    ("num", Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler()),
    ]), feature_cols)
])

def make_model(gridsearch=True):
    pipe = Pipeline([("pre", pre), ("ridge", Ridge())])
    if not gridsearch:
        return pipe
    param_grid = {
        "ridge__alpha": [0.01, 0.05, 0.1, 0.3, 1.0, 3.0, 10.0, 30.0],
        "ridge__fit_intercept": [True, False]
    }
    cv = TimeSeriesSplit(n_splits=5)
    grid = GridSearchCV(pipe, param_grid, cv=cv, scoring="neg_mean_absolute_error", n_jobs=-1)
    return grid

# ====================== GŁÓWNA PROGNOZA CO 6 MIESIĘCY OD 2020 ======================
plot_start = pd.Timestamp("2020-01-01")
max_anchor = df.index.max() - pd.DateOffset(months=6)
candidates = df.index[(df.index >= plot_start) & (df.index <= max_anchor)]
anchor_dates = list(candidates[::ANCHOR_STEP_MONTHS])

fan_segments = []
for anchor in anchor_dates:
    row = {"anchor": anchor, "p0": float(df.loc[anchor, "SOY"])}
    skip = False
    for h in HORIZONS_RECENT:
        y_col = f"SOY_t+{h}"
        cutoff = anchor - pd.DateOffset(months=h)
        df_h = df.dropna(subset=feature_cols + [y_col])
        train = df_h.loc[:cutoff]
        if len(train) < 36:
            skip = True
            break
        model = make_model(gridsearch=True)
        model.fit(train[feature_cols], train[y_col])
        X_anchor = df.loc[[anchor], feature_cols]
        row[f"f{h}"] = float(model.predict(X_anchor)[0])
        target_date = anchor + pd.DateOffset(months=h)
        row[f"t{h}"] = target_date
        row[f"a{h}"] = float(df.loc[target_date, "SOY"]) if target_date in df.index else np.nan
    if not skip:
        fan_segments.append(row)

fan_df = pd.DataFrame(fan_segments)

if not fan_df.empty:
    plt.figure(figsize=(13, 6))
    hist = df["SOY"].loc[plot_start: df.index.max()]
    plt.plot(hist.index, hist.values, color="black", linewidth=2, label="Actual")

    for i, row in fan_df.iterrows():
        xs = [row["anchor"]] + [row[f"t{h}"] for h in HORIZONS_RECENT]
        ys_model = [row["p0"]] + [row[f"f{h}"] for h in HORIZONS_RECENT]
        plt.plot(xs, ys_model, linestyle="--", linewidth=1.5, alpha=0.8,
                 label="Model forecast" if i == 0 else "")
        for h in HORIZONS_RECENT:
            if pd.notna(row[f"a{h}"]):
                plt.scatter([row[f"t{h}"]], [row[f"a{h}"]], marker="x", color="black", label="Actual" if i == 0 and h == 1 else "")
        plt.plot(xs, [row["p0"]] * len(xs), linestyle=":", alpha=0.5,
                 label="Persistence" if i == 0 else "")

    plt.title("SOY — forecast fans (2020+, co 6 mies., h=1..6)")
    plt.xlabel("Date"); plt.ylabel("USD/bushel")
    plt.legend(); plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()

    # ===== Tabela metryk trafnosci (model vs persistence) =====
    metric_rows = []
    for h in HORIZONS_RECENT:
        f_col, a_col = f"f{h}", f"a{h}"
        if f_col in fan_df.columns and a_col in fan_df.columns:
            valid = fan_df[[f_col, a_col, "p0"]].dropna()
            if len(valid) >= 3:
                y_true = valid[a_col].values
                y_pred = valid[f_col].values
                y_persist = valid["p0"].values
                metric_rows.append({
                    "h": h,
                    "N": len(valid),
                    "MAE_model": mae(y_true, y_pred),
                    "MAE_persist": mae(y_true, y_persist),
                    "R2_model": r2_score(y_true, y_pred),
                    "R2_persist": r2_score(y_true, y_persist)
                })
    metrics_df = pd.DataFrame(metric_rows).set_index("h")

    if not metrics_df.empty:
        print("\n=== Trafnosc prognoz (fan forecasts od 2020) ===")
        print(metrics_df.round(3))

        # Wykres MAE
        plt.figure(figsize=(8, 4))
        x = np.arange(len(metrics_df.index))
        width = 0.35
        plt.bar(x - width/2, metrics_df["MAE_model"], width, label="Model")
        plt.bar(x + width/2, metrics_df["MAE_persist"], width, label="Persistence")
        plt.xticks(x, [f"h={h}" for h in metrics_df.index])
        plt.ylabel("MAE")
        plt.title("MAE: model vs persistence")
        plt.legend()
        plt.grid(alpha=0.3)
        plt.tight_layout()
        plt.show()


Co jeszcze warto dodać jako zmienne (jeśli masz dane)

Podaż/popyt (USDA/CONAB/IGO):

Zapasy końcowe i stocks-to-use (USA/świat), produkcja, area harvested, plony (USA, Brazylia, Argentyna).

WASDE miesięczne rewizje (dummy „surprise” i/lub różnica vs konsensus).


Pogoda/uprawy:

Opady/temperatura anomalia (USA Midwest, Brasil/Matopiba) – miesięczne i 3M rolling.

SM/soil moisture, SWI (masz) + NDVI (wegetacja), GPP.

Faza ENSO (ONI/MEI), indeksy MJO, IOD – masz część (ONI, la_nina, enso_proxy).


Rynki i kursy:

FX BRL/USD (masz), DXY (masz), CNY (popyt z Chin).

WTI/Brent (biodiesel, oleje roślinne), Palm Oil (ceny zamienne), Corn/Wheat (substytuty).

Frachts: Baltic Dry/Grain freight (wpływ na ceny CIF/FOB).


Popyt i przetwórstwo:

COT (Commitments of Traders) – net positions Money Managers dla Soy (sentiment).

Crush margin / soy oil / soy meal spreads (relacje przetwórcze).

Ekspor t-y/w (US Export Inspections), chiński import (GACC), PMI China.


Struktura rynku:

Term structure: różnica front vs next futures (contango/backwardation).

Zmienność: 1M/3M vol z futures/returns (częściowo dodałem 3M vol).

Efekty kalendarzowe:

Dodatkowe dummies (np. miesiąc raportu WASDE, okres sadzenia/zbiorów).